## 簡潔版 Linear Regression

剛剛是自己動手寫各種Function. 在此介紹Tensorflow的內建Function.

## 生成數據集

In [1]:
import tensorflow as tf

num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = tf.random.normal(shape=(num_examples, num_inputs), stddev=1)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += tf.random.normal(labels.shape, stddev=0.01)

## 讀取數據

In [2]:
from tensorflow import data as tfdata

batch_size = 10
# 直接透過tensor餵入即可。
dataset = tfdata.Dataset.from_tensor_slices((features, labels))

dataset = dataset.shuffle(buffer_size=num_examples) # buffer size 要大於等於資料量。
dataset = dataset.batch(batch_size)
data_iter = iter(dataset)

In [6]:
next(data_iter)

(<tf.Tensor: id=44, shape=(10, 2), dtype=float32, numpy=
 array([[-0.6637609 , -0.8085974 ],
        [-1.2834051 , -1.196332  ],
        [ 0.48953813,  0.99965465],
        [-0.17394091, -1.692428  ],
        [-0.85391676, -0.62807006],
        [-1.097466  , -0.35862726],
        [-0.58516395,  0.79947895],
        [-0.53791046, -0.6792894 ],
        [ 0.758931  ,  0.0370138 ],
        [ 0.6877909 ,  0.18101417]], dtype=float32)>,
 <tf.Tensor: id=45, shape=(10,), dtype=float32, numpy=
 array([5.631625 , 5.698351 , 1.7889863, 9.628357 , 4.6288166, 3.224723 ,
        0.3062935, 5.421367 , 5.5969596, 4.9537306], dtype=float32)>)

In [7]:
for i in enumerate(dataset):
    print(i)
    break

(0, (<tf.Tensor: id=51, shape=(10, 2), dtype=float32, numpy=
array([[-0.69151026,  0.43541056],
       [-0.3707639 , -0.8472691 ],
       [-0.95836765, -0.5421561 ],
       [-0.8518849 ,  0.12394872],
       [ 0.76590306, -1.2388538 ],
       [-0.01132818,  0.95249385],
       [-1.0080208 , -1.64112   ],
       [-0.6822649 ,  0.10336093],
       [-0.6940032 , -0.6383262 ],
       [ 0.85116905, -0.21288364]], dtype=float32)>, <tf.Tensor: id=52, shape=(10,), dtype=float32, numpy=
array([1.3330171 , 6.3538446 , 4.128469  , 2.081712  , 9.951288  ,
       0.92697877, 7.7580466 , 2.478191  , 4.967567  , 6.6169662 ],
      dtype=float32)>))


## 建構Model

In [8]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow import initializers as init
model = keras.Sequential()
model.add(layers.Dense(1, kernel_initializer=init.RandomNormal(stddev=0.01)))

## 定義損失函數

In [16]:
from tensorflow import losses
loss = losses.MeanSquaredError()

## 定義優化方法

In [17]:
from tensorflow.keras import optimizers
trainer = optimizers.SGD(learning_rate=0.03)

## 設計整個model flow

In [18]:
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for (batch, (X, y)) in enumerate(dataset):
        with tf.GradientTape() as tape:
            l = loss(model(X, training=True), y)

        grads = tape.gradient(l, model.trainable_variables)
        trainer.apply_gradients(zip(grads, model.trainable_variables))

    l = loss(model(features), labels)
    print('epoch %d, loss: %f' % (epoch, l.numpy().mean()))

epoch 1, loss: 0.000217
epoch 2, loss: 0.000100
epoch 3, loss: 0.000100


In [29]:
true_w, model.get_weights()[0]

([2, -3.4], array([[ 1.9996798],
        [-3.4005053]], dtype=float32))